In [1]:
import numpy as np
from stochastic_solvers_imagenet import *
from utils import *
from data_utils import *
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode
from torchvision import transforms
from tqdm import tqdm

np.random.seed(41)

In [2]:
batch_size = 64

In [3]:
train_dir = "ILSVRC/Data/CLS-LOC/train"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.view(-1)),  
    transforms.Lambda(lambda x: x / 255.0)
])

dataset = ImageNetTrainDataset(train_dir, transform=transform)

randomBatchGetter = RandomBatchGetter(dataset, batch_size)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 580.57it/s]


In [4]:
r = 30
step_size_func = lambda idx: 2 / (1 + np.floor(idx / 100))

# eval_func = lambda x: compute_avg_error(true_evecs, x)

In [5]:
import time

start_time = time.time()
L = 50
T = 10

evecs_hist = parallel_deflation_1(r, L, T, step_size_func, randomBatchGetter, update='hebb', num_workers=16 * 2, parallel_mode="thread")

print(f"Time taken {time.time() - start_time}")

Current round 0
Current round 1
Current round 2
Current round 3
Current round 4
Time taken 17.874545097351074


In [9]:
import time

start_time = time.time()
L = 50
T = 10

final_evecs = parallel_deflation_save_only_last(r, L, T, step_size_func, randomBatchGetter, update='hebb', num_workers=16 * 2, parallel_mode="thread")

print(f"Time taken {time.time() - start_time}")

Current round 0
Current round 1
Current round 2
Current round 3
Current round 4
Time taken 16.754937887191772


In [10]:
final_evecs.shape

(30, 50176)

# Start from Here!

In [1]:
import numpy as np
from stochastic_solvers_imagenet import *
from utils import *
from data_utils import *
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode
from torchvision import transforms
from tqdm import tqdm

np.random.seed(41)

import time

train_dir = "ILSVRC/Data/CLS-LOC/train"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.view(-1)),  
    transforms.Lambda(lambda x: x / 255.0)
])

batch_size = 1000
dataset = ImageNetTrainDataset(train_dir, transform=transform)
randomBatchGetter = RandomBatchGetter(dataset, batch_size)

eval_batch_size = 1000
eval_randomBatchGetter = RandomBatchGetter(dataset, eval_batch_size)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 578.36it/s]


In [ ]:
start_time = time.time()
r = 30
L = 50
T = 10
# step_size_func = lambda idx: 2 / (1 + np.floor(idx / 100))

step_size_func = decaying_schedule_with_warmup

final_evecs = parallel_deflation_save_only_last(r, L, T, step_size_func, 
                                                randomBatchGetter, 
                                                update='hebb', 
                                                eval_func=evaluate_ImageNet, 
                                                eval_B=eval_batch_size, 
                                                eval_randomBatchGetter=eval_randomBatchGetter, 
                                                num_workers=10, 
                                                parallel_mode="thread")

print(f"Time taken {time.time() - start_time}")

Current round 0
Computing for lambda 0


 22%|████████████████████████████████▎                                                                                                                  | 281/1280 [10:09<36:14,  2.18s/it]